# **Running Jupyter Code with Ollama Server**

1. **Download and Install Ollama**: Ensure that Ollama is downloaded and installed on your system.

2. **Check Ollama Version**:
   ```bash
   ollama --version
   ```

3. **Run Llama 3.2 Model**:
   To load the Llama 3.2 model:
   ```bash
   ollama run llama3.2
   ```

4. **Start Ollama Server**:
   To start the Ollama server, run:
   ```bash
   ollama serve
   ```

**Note**: If you face any issues while running the server, checking the background tasks using the port can help resolve conflicts. Always ensure the port is free before starting the server.

1. **Check if Ollama Server is Running**:
   Check if the server is running and using port 11434:
   ```bash
   netstat -ano | findstr :11434
   ```

2. **Identify and Kill Background Task (if needed)**:
   If you see that another task is using the port, you can identify and kill it:
   ```bash
   tasklist /FI "PID eq <PID>"
   taskkill /PID <PID> /F
   ```

3. **Close Ollama from Windows Task Bar (Optional)**:
   If the server is still not responding or you see issues, you can use the **up arrow key** (^) on the Windows task bar (bottom-left) to close any running Ollama instances.

4. **Start Jupyter Notebook**:
   Once the server is running, start your Jupyter notebook with the necessary code to interact with Ollama.

In [21]:
import ollama
from PyPDF2 import PdfReader

In [22]:
system_prompt = '''You are an agent which analyses the resume and understands the context of resume and 
    extract the following fields and format the output:
    - Name
    - Email
    - Phone 1
    - Phone 2
    - Address
    - City
    - LinkedIn
    - Professional Experience (in years)
    - Highest Education
    - Is Fresher (yes/no)
    - Is Student (yes/no)
    - Skills (comma-separated)
    - Applied For Profile
    - Education (Institute Name, Year of Passing, Score)
    - Projects (Project Name, Description)
    - Professional Experience (Organisation Name, Duration, Profile)

    Ensure to format the output as:
    Name: [Value]
    Email: [Value]
    Phone 1: [Value]
    ... and provide this summary of resume in markdown format'''

In [23]:
def extract_text_from_pdf(pdf_path):
    text = ""
    try:
        with open(pdf_path, "rb") as file:
            reader = PdfReader(file)
            for page in reader.pages:
                text += page.extract_text()  # Extract text from each page
    except Exception as e:
        print(f"Error extracting text from PDF: {e}")
    return text
    
user_prompt = extract_text_from_pdf('./data/python-developer-resume-example.pdf') + "Provide a summary of this resume"

In [24]:
import requests
import json

url = "http://localhost:11434/api/chat"
headers = {
    "Content-Type": "application/json"
}

# Define the conversation prompt and specify the model
data = {
    "model": "llama3.2",  # Specify the model name here
    "messages": [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]
}

# Send the POST request to the Ollama server
response = requests.post(url, json=data, headers=headers, stream=True)

# Check if the response is successful
if response.status_code == 200:
    # Initialize a variable to accumulate the assistant's response
    full_response = ""
    
    # Iterate over the chunks of data as they arrive
    for chunk in response.iter_lines():
        if chunk:  # Ignore empty lines
            try:
                # Each chunk is a part of the message, so we can parse it as JSON
                chunk_data = json.loads(chunk.decode('utf-8'))
                
                # Extract the message content
                message_content = chunk_data.get('message', {}).get('content', '')
                full_response += message_content
                
                # Check if the response is done
                if chunk_data.get('done', False):
                    break
                
            except json.JSONDecodeError as e:
                print("Error parsing JSON chunk:", e)
    
    print("Model Response:", full_response)
else:
    print(f"Error {response.status_code}: {response.text}")


Model Response: ### Resume Summary

#### Contact Information

* **Name:** G I U L I A
* **Email:** `ggonzalez@email.com`
* **Phone 1:** `(123) 456-7890`

#### Address and Social Media

* **Address:** Detroit, MI
* **City:** Detroit, MI
* **LinkedIn:** LinkedIn
* **Github:** Github

#### Education

* **Highest Education:** M.S. in Computer Science from University of Chicago (2014 - 2016)
* **Previous Education:**
	+ B.S. in Computer Science from University of Pittsburgh (2010 - 2014)

#### Work Experience and Skills

* **Professional Experience:** Python Developer
	+ Duration: September 2017 - current at DoorDash (Detroit, MI)
	+ Summary:
		- Improved time on page for the average user by 2 minutes
		- Reduced customer complaints by 23%
		- Boosted revenue by 6%
* **Skills:** HTML/ CSS, SQL (PostgreSQL, Oracle), JavaScript (Angular), Python (Django), REST APIs (GraphQL), AWS (Redshift, S3), Git

#### Work Experience Details

* **Python Developer Intern**
	+ Institution: Knewton
	+ Durati